<a href="https://colab.research.google.com/github/Mehdi-charfi/face-extractor/blob/master/face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import torch
import torch.nn as nn
import dlib
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import shutil

In [ ]:
# Load pre-trained models for face and upper body detection
face_detector = dlib.get_frontal_face_detector()
upper_body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')


In [ ]:
# Load pre-trained feature extraction models (e.g., ResNet50)
from torchvision.models import ResNet50_Weights

face_feature_extractor = models.resnet50(weights=ResNet50_Weights.DEFAULT)
face_feature_extractor.fc = nn.Identity()  # Remove the final classification layer
face_feature_extractor.eval()  # Set the model to evaluation mode

upper_body_feature_extractor = models.resnet50(weights=ResNet50_Weights.DEFAULT)
upper_body_feature_extractor.fc = nn.Identity()  # Remove the final classification layer
upper_body_feature_extractor.eval()  # Set the model to evaluation mode


In [ ]:

# Define image preprocessing function
def preprocess_image(image, target_size=(224, 224)):
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize to [0, 1]
    image = np.transpose(image, (2, 0, 1))  # Convert HWC to CHW format
    return torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Function to detect, resize, and save faces and upper bodies, and return their embeddings
def detect_and_extract_embeddings(image_path, save_crops=False, output_dir="detected_crops", crop_size=(128, 128)):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces and upper bodies
    faces = face_detector(gray)
    upper_bodies = upper_body_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(60, 60))

    # Create output directory if saving crops
    if save_crops and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    embeddings = []
       # Process faces
    for idx, face in enumerate(faces):
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_crop = image[y:y+h, x:x+w]
        face_resized = cv2.resize(face_crop, crop_size)

        # Save face crop if required
        if save_crops:
            face_filename = os.path.join(output_dir, f"face_{os.path.basename(image_path).split('.')[0]}_{idx}.png")
            cv2.imwrite(face_filename, face_resized)

        # Extract face embeddings
        face_tensor = preprocess_image(face_resized)
        with torch.no_grad():
            face_embedding = face_feature_extractor(face_tensor).squeeze().numpy()
        embeddings.append((face_embedding, "face", face_filename if save_crops else image_path))

    # Process upper bodies
    for idx, (x, y, w, h) in enumerate(upper_bodies):
        upper_body_crop = image[y:y+h, x:x+w]
        upper_body_resized = cv2.resize(upper_body_crop, crop_size)

        # Save upper body crop if required
        if save_crops:
            upper_body_filename = os.path.join(output_dir, f"upper_body_{os.path.basename(image_path).split('.')[0]}_{idx}.png")
            cv2.imwrite(upper_body_filename, upper_body_resized)

        # Extract upper body embeddings
        upper_body_tensor = preprocess_image(upper_body_resized)
        with torch.no_grad():
            upper_body_embedding = upper_body_feature_extractor(upper_body_tensor).squeeze().numpy()
        embeddings.append((upper_body_embedding, "upper_body", upper_body_filename if save_crops else image_path))

    return embeddings


In [ ]:

# Define data augmentation pipeline
augmentation_pipeline = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image_path

# Custom collate function to handle batch issues
def custom_collate_fn(batch):
    images, paths = zip(*batch)
    return list(images), list(paths)

# Function to load image paths from a directory
def load_image_paths(directory):
    return [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.lower().endswith(('png', 'jpg', 'jpeg'))]


In [ ]:

# Function to find the optimal number of clusters using the silhouette score
def find_optimal_clusters(embeddings):
    best_num_clusters = 2
    best_score = -1
    for num_clusters in range(2, min(11, len(embeddings))):  # Limit to 10 or fewer clusters for efficiency
        clustering_model = AgglomerativeClustering(n_clusters=num_clusters)
        labels = clustering_model.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        if score > best_score:
            best_num_clusters = num_clusters
            best_score = score
    return best_num_clusters

# Function to cluster embeddings and save images in classified folders
def cluster_embeddings(embeddings_with_filenames, output_dir="classified_crops"):
    embeddings = [embedding for embedding, _, _ in embeddings_with_filenames]
    optimal_clusters = find_optimal_clusters(embeddings)
    clustering_model = AgglomerativeClustering(n_clusters=optimal_clusters)
    labels = clustering_model.fit_predict(embeddings)

    # Create output directories for each cluster
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for cluster_idx in range(optimal_clusters):
        cluster_dir = os.path.join(output_dir, f"cluster_{cluster_idx}")
        if not os.path.exists(cluster_dir):
            os.makedirs(cluster_dir)

    # Save images to respective cluster folders
    for (_, crop_type, filename), label in zip(embeddings_with_filenames, labels):
        shutil.copy(filename, os.path.join(output_dir, f"cluster_{label}", f"{crop_type}_{os.path.basename(filename)}"))

    return labels



In [ ]:
# Main function
if __name__ == "__main__":
    # Path to your image directory
    image_directory = "imgs"

    # Load image paths
    image_paths = load_image_paths(image_directory)

    # Create dataset and data loader
    image_dataset = CustomImageDataset(image_paths=image_paths, transform=augmentation_pipeline)
    data_loader = DataLoader(image_dataset, batch_size=32, shuffle=True, num_workers=3, collate_fn=custom_collate_fn)  # Reduced num_workers for system compatibility

    # Collect features and filenames for clustering
    all_embeddings_with_filenames = []

    # Iterate over batches and process
    for batch, paths in data_loader:
        for i, path in enumerate(paths):
            embeddings = detect_and_extract_embeddings(path, save_crops=True)
            all_embeddings_with_filenames.extend(embeddings)

    # Perform clustering on the collected embeddings
    labels = cluster_embeddings(all_embeddings_with_filenames)

    # Print clustering results
    print("Clustering complete! Faces and upper bodies have been classified and saved in respective cluster folders.")


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Clustering complete! Faces and upper bodies have been classified and saved in respective cluster folders.
